In [40]:
import pickle
import nltk
import math

In [41]:
class Docs:

    def __init__(self,Lancaster=True,split=False):

        docs = []
        self.maxfreq = 0
        MotsVides = nltk.corpus.stopwords.words('english')

        self.files = ["TP2data/D1.txt","TP2data/D2.txt","TP2data/D3.txt","TP2data/D4.txt","TP2data/D5.txt","TP2data/D6.txt"]
        file = ""
        my_dict = {}
        for i in range(len(self.files)):
            file = open(self.files[i]).read()
            file = file.replace("\n","")

            if split :
                TermesNormalisation = file.split()
            else :

                ExpReg = nltk.RegexpTokenizer('(?:[A-Z]\.)+|\w+') # \d : équivalent à [0-9] >>> 
                Termes = ExpReg.tokenize(file) 

            if Lancaster :
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                Lancaster = nltk.LancasterStemmer()
                TermesNormalisation = [Lancaster.stem(terme) for terme in TermesSansMotsVides]
            else :
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                porter = nltk.PorterStemmer()
                TermesNormalisation = [porter.stem(terme) for terme in TermesSansMotsVides]


            for term in TermesNormalisation:
                
                if (term,i) in my_dict:
                    my_dict[(term,i)] += 1
                else:
                    my_dict[(term,i)] = 1

        self.dict = my_dict

        for keys in self.dict.keys():
            if keys[1] not in docs:
                docs.append(keys[1])
            if self.dict[keys] > self.maxfreq:
                self.maxfreq = self.dict[keys]
        print(self.maxfreq)
        self.N = len(docs)
    
    def rebuild(self,Lancaster=True,split=False):
        docs = []
        self.maxfreq = 0
        MotsVides = nltk.corpus.stopwords.words('english')

        
        file = ""
        my_dict = {}
        for i in range(len(self.files)):
            file = open(self.files[i]).read()
            file = file.replace("\n","")


            if not split :
                Termes = file.split()
            else :
                ExpReg = nltk.RegexpTokenizer('(?:[A-Z]\.)+|\w+|\.{3}') # \d : équivalent à [0-9] >>> 
                Termes = ExpReg.tokenize(file) 
            

            if Lancaster :
                print('lancaster')
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                Lancaster = nltk.LancasterStemmer()
                TermesNormalisation = [Lancaster.stem(terme) for terme in TermesSansMotsVides]
            else :
                print('porter')
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                porter = nltk.PorterStemmer()
                TermesNormalisation = [porter.stem(terme) for terme in TermesSansMotsVides]

            for term in TermesNormalisation:
                
                if (term,i) in my_dict:
                    my_dict[(term,i)] += 1
                else:
                    my_dict[(term,i)] = 1

        self.dict = my_dict

        for keys in self.dict.keys():
            if keys[1] not in docs:
                docs.append(keys[1])
            if self.dict[keys] > self.maxfreq:
                self.maxfreq = self.dict[keys]

        self.N = len(docs)
        self.DictWeight()

    def calcWeight(self,word,doc):

        inDocs = []

        for keys in self.dict.keys():
            if keys[0] == word:
                if keys[1] not in inDocs:
                    inDocs.append(keys[1])

        # nombre de documents contenant le mot
        Ni = len(inDocs)

        try:
            tf = self.dict[(word,doc)]/self.maxfreq
        except:
            tf = 0
            
        #idf = math.log10((N/Ni)+1)
        try:
            idf = math.log10((self.N/Ni)+1)
        except:
            idf = 0

        return(tf*idf)

    def DictWeight(self):
        self.TDFP = [(word,doc,self.dict[(word,doc)],self.calcWeight(word,doc)) for word,doc in self.dict.keys()]
        self.DTFP = [(doc,word,freq,weight) for word,doc,freq,weight in self.TDFP]
        self.TDFP = sorted(self.TDFP, key=lambda x: x[0])

        
d = Docs(Lancaster=False)

d.DictWeight()

d.TDFP

<>:20: SyntaxWarning: invalid escape sequence '\.'
<>:66: SyntaxWarning: invalid escape sequence '\.'
<>:20: SyntaxWarning: invalid escape sequence '\.'
<>:66: SyntaxWarning: invalid escape sequence '\.'
C:\Users\mekki\AppData\Local\Temp\ipykernel_6704\4079350013.py:20: SyntaxWarning: invalid escape sequence '\.'
  ExpReg = nltk.RegexpTokenizer('(?:[A-Z]\.)+|\w+') # \d : équivalent à [0-9] >>>
C:\Users\mekki\AppData\Local\Temp\ipykernel_6704\4079350013.py:66: SyntaxWarning: invalid escape sequence '\.'
  ExpReg = nltk.RegexpTokenizer('(?:[A-Z]\.)+|\w+|\.{3}') # \d : équivalent à [0-9] >>>


11


[('0', 4, 1, 0.07682709454675062),
 ('000', 2, 1, 0.07682709454675062),
 ('04', 5, 1, 0.07682709454675062),
 ('10', 4, 2, 0.15365418909350123),
 ('100', 2, 2, 0.15365418909350123),
 ('17', 2, 1, 0.07682709454675062),
 ('2', 4, 1, 0.07682709454675062),
 ('2014', 1, 1, 0.07682709454675062),
 ('2019', 0, 1, 0.07682709454675062),
 ('2020', 0, 1, 0.07682709454675062),
 ('3', 0, 1, 0.07682709454675062),
 ('4', 0, 1, 0.043374659519969314),
 ('4', 2, 1, 0.043374659519969314),
 ('4', 4, 1, 0.043374659519969314),
 ('5', 0, 1, 0.07682709454675062),
 ('500x', 2, 1, 0.07682709454675062),
 ('6', 4, 1, 0.07682709454675062),
 ('7b', 0, 1, 0.07682709454675062),
 ('abil', 2, 1, 0.07682709454675062),
 ('achiev', 0, 1, 0.07682709454675062),
 ('addit', 1, 1, 0.043374659519969314),
 ('addit', 2, 1, 0.043374659519969314),
 ('addit', 5, 1, 0.043374659519969314),
 ('address', 0, 1, 0.03617636442473069),
 ('address', 1, 1, 0.03617636442473069),
 ('address', 2, 1, 0.03617636442473069),
 ('address', 5, 1, 0.03617

In [42]:
import tkinter as tk
from tkinter import ttk

def search():

    text = query_text.get()
    table.delete(*table.get_children())

    if index_var.get() == 0:
        data = d.TDFP
    else:
        data = d.DTFP

    for terme in data:

        if type(terme[0]) is str:
            if text.lower() in terme[0].lower():
                table.insert("", "end", values=[terme[0],terme[1],terme[2],terme[3]])
        else:
            if text.lower() in terme[1].lower():
                table.insert("", "end", values=[terme[0],terme[1],terme[2],terme[3]])
    # Add your search and processing logic here
    pass

def on_token_change(*args):
    d.rebuild(Lancaster=stemming.get(),split=token.get())
    search()

def on_index_change(*args):
    search()

# Create the main application window
root = tk.Tk()
root.title("Text Processing Application")

# Search Query Text Box and Search Button
query_label = tk.Label(root, text="Search Query:")
query_label.grid(row=0, column=0)
query_text = tk.Entry(root, width=40)
query_text.grid(row=0, column=1)
search_button = tk.Button(root, text="Search", command=search)
search_button.grid(row=0, column=2)

# Border around "Processing" section
processing_label_frame = ttk.LabelFrame(root, text="Processing")
processing_label_frame.grid(row=1, column=2, padx=10, pady=10)

# Radio buttons for Processing
stemming = tk.IntVar()
stemming.trace("w", on_token_change)

token = tk.IntVar()
token.trace("w", on_token_change)

tokenization_radiobutton = tk.Checkbutton(processing_label_frame, text="Tokenization", variable=token)
porter_stemmer_radiobutton = tk.Checkbutton(processing_label_frame, text="Lancaster Stemmer", variable=stemming)
porter_stemmer_radiobutton.select()
tokenization_radiobutton.grid(row=0, column=0)
porter_stemmer_radiobutton.grid(row=1, column=0)

# Border around "Index" section
index_label_frame = ttk.LabelFrame(root, text="Index")
index_label_frame.grid(row=1, column=0, padx=10, pady=10)

# Radio buttons for Index

index_var = tk.IntVar()
index_var.trace("w", on_index_change)

docs_per_term_radiobutton = tk.Radiobutton(index_label_frame, text="DOCS per TERM", variable=index_var, value=0)
terms_per_doc_radiobutton = tk.Radiobutton(index_label_frame, text="TERMS per DOC", variable=index_var, value=1)
docs_per_term_radiobutton.grid(row=1, column=0)
terms_per_doc_radiobutton.grid(row=2, column=0)

# Table to display data
table_frame = ttk.Frame(root)
table_frame.grid(row=2, column=0, columnspan=3, pady=10)
table = ttk.Treeview(table_frame, columns=("1", "2", "3", "4"))
table.heading("#1", text="terme")
table.heading("#2", text="document")
table.heading("#3", text="freq")
table.heading("#4", text="Score")
table['show'] = 'headings'
# Vertical scrollbar for the table
table_scrollbar = ttk.Scrollbar(table_frame, orient="vertical", command=table.yview)
table.configure(yscrollcommand=table_scrollbar.set)

# Grid layout for the table and scrollbar
table.grid(row=0, column=0, sticky="nsew")
table_scrollbar.grid(row=0, column=1, sticky="ns")


# Make the table resizable
table_frame.grid_rowconfigure(0, weight=1)
table_frame.grid_columnconfigure(0, weight=1)


for dat in d.TDFP:
    table.insert("", "end", values=[dat[0],dat[1],dat[2],dat[3]])


root.mainloop()

lancaster
lancaster
lancaster
lancaster
lancaster
lancaster


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\mekki\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\mekki\AppData\Local\Temp\ipykernel_6704\2791770733.py", line 27, in on_token_change
    search()
  File "C:\Users\mekki\AppData\Local\Temp\ipykernel_6704\2791770733.py", line 7, in search
    table.delete(*table.get_children())
                  ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mekki\AppData\Local\Programs\Python\Python312\Lib\tkinter\ttk.py", line 1195, in get_children
    self.tk.call(self._w, "children", item or '') or ())
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
_tkinter.TclError: invalid command name ".!frame.!treeview"
